# 网易考拉口红销售数据可视化以及数据分析探索

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False

## 数据导入、查看

In [8]:
# 查看当前挂载的数据集目录
Data=pd.read_excel(r"D:\PycharmProjects\Test_learning\lipsticks\lipsticks.xlsx")

In [9]:
#查看数据集基本情况
Data.head()

,品牌,页面网址,采集时间,折扣价,原价,商品标题,是否自营,标签1,标签2,标签3,评论数,商品链接,国家,店铺
0,Dior 迪奥,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥198,¥299,Dior 迪奥粉漾魅惑变色润唇膏 01#粉色3.5克,自营,白条免息,NaN,NaN,23440,https://www.kaola.com/product/1588979.html,法国,网易考拉自营
1,Dior 迪奥,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥298,¥320,Dior 迪奥 烈艳蓝金唇膏烈焰口红 3.5克,自营,包税,白条免息,NaN,4350,https://www.kaola.com/product/2415785.html,法国,网易考拉自营
2,Dior 迪奥,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥169,NaN,【双11预售】Dior 迪奥 粉漾魅惑变色润唇膏 3.5克,自营,白条免息,NaN,NaN,555,https://www.kaola.com/product/2053434.html,法国,网易考拉自营
3,Dior 迪奥,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥179,¥300,Dior 迪奥 魅惑釉唇膏 3.2克,自营,白条免息,特价,NaN,1690,https://www.kaola.com/product/1600086.html,法国,网易考拉自营
4,Dior 迪奥,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥169,¥300,Dior 迪奥 粉漾魅惑变色润唇膏 3.5克,自营,白条免息,特价,NaN,545,https://www.kaola.com/product/2320261.html,法国,网易考拉自营


In [10]:
Data.dtypes

品牌      object
页面网址    object
采集时间    object
折扣价     object
原价      object
商品标题    object
是否自营    object
标签1     object
标签2     object
标签3     object
评论数      int64
商品链接    object
国家      object
店铺      object
dtype: object

In [11]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 14 columns):
品牌      540 non-null object
页面网址    540 non-null object
采集时间    540 non-null object
折扣价     540 non-null object
原价      525 non-null object
商品标题    540 non-null object
是否自营    455 non-null object
标签1     525 non-null object
标签2     326 non-null object
标签3     178 non-null object
评论数     540 non-null int64
商品链接    540 non-null object
国家      540 non-null object
店铺      455 non-null object
dtypes: int64(1), object(13)
memory usage: 59.1+ KB


通过对数据的简单描述发现：
总共有14个特征列，每个特征列有540个数据。
有6列有一定程度的缺失（原价/是否自营/标签1/标签2/标签3/店铺）

## 数据清洗
缺失值
重复值

In [12]:
# 查看重复
Data.duplicated(['商品标题','折扣价','是否自营']).count()#主要指标无重复

540

In [13]:
# 缺失值处理 :是否自营和店铺均填充为非自营,原价设置为折扣价
Data['是否自营'].fillna(value='非自营',inplace=True)
Data['店铺'].fillna(value='非自营',inplace=True)
Data['原价'].fillna(value=Data['折扣价'],inplace=True)

In [14]:
# 浏览修改后的数据发现原价中含有“新人价”、“XX折” 新人价用折扣价填充，XX折则与折扣计算后填充首先将新人价替换成折扣价
Data['price_0'] =Data['原价'].replace('新人价',Data['折扣价'])

In [15]:
discount_list=Data.loc[Data['price_0'].str.contains('折'),"原价"].values# 筛选原价列中含有”折“的行，并转化为列表
price_list=Data.loc[Data['price_0'].str.contains('折'),"折扣价"].values# 筛选折扣价列中含有”折“的行，并转化为列表
print(discount_list,price_list)


['6.7折' '9.6折' '8.9折' '9.8折' '7.8折' '7折' '7折' '7.5折' '6.4折' '7.8折' '7.1折'
 '6.4折' '5.9折' '9.1折' '4.2折' '3.3折' '6.5折' '6.2折' '7折' '8.5折' '7.6折'
 '8.7折' '8折' '5.9折' '8.4折' '5.9折' '7折' '8.3折' '8.8折' '4.4折' '6.9折' '8.3折'
 '6.6折' '8.1折' '8.6折' '8.4折' '6.6折' '8折' '8折' '3.4折' '3.5折' '8.1折' '8.3折'
 '8.3折' '7.4折' '3.8折' '6.6折' '6.5折' '6折' '6.6折' '6.6折' '7.1折' '6.6折' '6折'
 '6.6折' '6.6折' '6.6折' '6.6折' '6.6折' '6.6折' '6.6折' '7.1折' '6折' '5.1折' '6折'] ['¥302' '¥308' '¥285' '¥315' '¥109' '¥119' '¥119' '¥119' '¥119' '¥109'
 '¥125' '¥119' '¥109' '¥99' '¥79' '¥99' '¥104' '¥49' '¥118' '¥118' '¥75'
 '¥69' '¥79' '¥35' '¥75' '¥175' '¥118' '¥99' '¥105' '¥75' '¥89' '¥99'
 '¥99' '¥105' '¥268' '¥268' '¥19' '¥39' '¥39' '¥119' '¥99' '¥143' '¥166'
 '¥170' '¥238' '¥246' '¥39' '¥129' '¥119' '¥39' '¥39' '¥42' '¥39' '¥59'
 '¥39' '¥39' '¥39' '¥39' '¥39' '¥39' '¥39' '¥42' '¥59' '¥29.9' '¥59']


In [16]:
value_list=[]#将该部分行提取折扣并转换为列表
for i,j in zip(discount_list,price_list):
    i=float(i.replace('折',''))
    j=float(j.replace('¥', ''))
    z=round(j * 10 / i,2)
    z='¥'+str(z)
    value_list.append(z)
print(value_list)

['¥450.75', '¥320.83', '¥320.22', '¥321.43', '¥139.74', '¥170.0', '¥170.0', '¥158.67', '¥185.94', '¥139.74', '¥176.06', '¥185.94', '¥184.75', '¥108.79', '¥188.1', '¥300.0', '¥160.0', '¥79.03', '¥168.57', '¥138.82', '¥98.68', '¥79.31', '¥98.75', '¥59.32', '¥89.29', '¥296.61', '¥168.57', '¥119.28', '¥119.32', '¥170.45', '¥128.99', '¥119.28', '¥150.0', '¥129.63', '¥311.63', '¥319.05', '¥28.79', '¥48.75', '¥48.75', '¥350.0', '¥282.86', '¥176.54', '¥200.0', '¥204.82', '¥321.62', '¥647.37', '¥59.09', '¥198.46', '¥198.33', '¥59.09', '¥59.09', '¥59.15', '¥59.09', '¥98.33', '¥59.09', '¥59.09', '¥59.09', '¥59.09', '¥59.09', '¥59.09', '¥59.09', '¥59.15', '¥98.33', '¥58.63', '¥98.33']


In [17]:
pd.DataFrame(value_list).shape

(65, 1)

In [18]:
Data.loc[Data['price_0'].str.contains('折'),"price_0"]=value_list

In [19]:
Data["price_0"].value_counts().head()

¥320    170
¥300     24
¥195     22
¥170     19
¥355     17
Name: price_0, dtype: int64

In [20]:
Data['折扣价_0']=Data['折扣价'].str.replace('¥', '').astype(float)

Data['原价_0']=Data['price_0'].str.replace('¥', '').astype(float)

Data['discount']=Data['折扣价_0']/Data['原价_0']

In [21]:
Data.tail(10)

,品牌,页面网址,采集时间,折扣价,原价,商品标题,是否自营,标签1,标签2,标签3,评论数,商品链接,国家,店铺,price_0,折扣价_0,原价_0,discount
530,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥39,6.6折,【江疏影同款 袁姗姗剧款 水瓶座专属 包邮】Manuka bee 小蜜坊 亲蜜润唇棒 5.5...,非自营,79选3,NaN,NaN,15,https://www.kaola.com/product/2476322.html,新西兰,非自营,¥59.09,39.0,59.09,0.660010
531,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥39,6.6折,【江疏影同款 袁姗姗剧款 金牛座专属 包邮】Manuka bee 小蜜坊 亲蜜润唇棒 5.5...,非自营,79选3,NaN,NaN,14,https://www.kaola.com/product/2475930.html,新西兰,非自营,¥59.09,39.0,59.09,0.660010
532,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥39,6.6折,【江疏影同款 袁姗姗剧款 白羊座专属 包邮】Manuka bee 小蜜坊 亲蜜润唇棒 5.5...,非自营,79选3,NaN,NaN,14,https://www.kaola.com/product/2475741.html,新西兰,非自营,¥59.09,39.0,59.09,0.660010
533,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥39,6.6折,【江疏影同款 袁姗姗剧款 双子座专属 包邮】Manuka bee 小蜜坊 亲蜜润唇棒 5.5...,非自营,79选3,NaN,NaN,10,https://www.kaola.com/product/2475781.html,新西兰,非自营,¥59.09,39.0,59.09,0.660010
534,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥39,6.6折,【江疏影同款 袁姗姗剧款 射手座专属 包邮】Manuka bee 小蜜坊 亲蜜润唇棒 5.5...,非自营,79选3,NaN,NaN,7,https://www.kaola.com/product/2476281.html,新西兰,非自营,¥59.09,39.0,59.09,0.660010
535,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥39,6.6折,【江疏影同款 袁姗姗剧款 处女座专属 包邮】Manuka bee 小蜜坊 亲蜜润唇棒 5.5...,非自营,79选3,NaN,NaN,5,https://www.kaola.com/product/2476062.html,新西兰,非自营,¥59.09,39.0,59.09,0.660010
536,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥42,7.1折,【江疏影同款】【双鱼座专属 包邮】Manuka bee 小蜜坊 亲蜜润唇棒 5.5g 薄荷绿,非自营,79选3,NaN,NaN,6,https://www.kaola.com/product/2476378.html,新西兰,非自营,¥59.15,42.0,59.15,0.710059
537,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥59,6折,【双色玩转 护唇新体验 包邮】Manuka Bee 小蜜坊 变色润唇棒 珊瑚红,非自营,特价,特价,NaN,3,https://www.kaola.com/product/2544041.html,新西兰,非自营,¥98.33,59.0,98.33,0.600020
538,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥29.9,5.1折,【江疏影同款 袁姗姗剧款 十二星座专属 】Manuka bee 小蜜坊 亲蜜润唇棒 5.5g,非自营,特价,特价,NaN,0,https://www.kaola.com/product/2680449.html,新西兰,非自营,¥58.63,29.9,58.63,0.509978
539,Manuka Bee 小蜜坊,https://www.kaola.com/search.html?key=%25E5%25...,2018-11-08,¥59,6折,【唇部SPA 包邮】Manuka bee 小蜜坊 三重密集滋养修护唇膜,非自营,特价,特价,NaN,20,https://www.kaola.com/product/2499878.html,新西兰,非自营,¥98.33,59.0,98.33,0.600020


## 品牌

### 热度

In [22]:
from pyecharts import configure
configure(global_theme='vintage')
from pyecharts import Line, Bar, Pie,Grid,WordCloud,Overlap
from IPython.display import display, Image
x=Data.groupby('品牌').sum()['评论数'].sort_values(ascending=False).index
y=Data.groupby('品牌').sum()['评论数'].sort_values(ascending=False)

In [27]:
bar=Bar(title='品牌VS评论数总和', subtitle="2019-09-22",title_pos='left',title_text_size=22,width=1000,height=800,background_color='white')
bar.add(name='品牌VS评论数总和',x_axis =x, y_axis = y,bar_category_gap=5, is_label_show=True, yaxis_name= "评论数总和",xaxis_line_width= 10,
    xaxis_rotate = 30,xaxis_label_textsize='9',yaxis_label_textsize='9')
display(bar)

### 原产地

In [ ]:
from pyecharts import Pie
configure(output_image=True)
attr = Data['国家'].value_counts().sort_values(ascending=False).index
v=Data['国家'].value_counts().sort_values(ascending=False)
pie =Pie('各产地国家占比',background_color = 'white',title_pos = 'center')
pie.add('',attr,v,is_label_show = True,radius=[25,75],is_legend_show =False,label_formatter='{d}')
display(pie)

可以看到在网易考拉网上，迪奥的市场份额属于第一梯队，资生堂、MAC、圣罗兰、纪梵希属于第二梯队；
热门品牌中原产地多数为法国，原产地的饼图也可见法国为口红大国，占了52.22%。

### SKU

In [ ]:
x1=Data['品牌'].value_counts().sort_values(ascending=False).index
y1=Data['品牌'].value_counts().sort_values(ascending=False)
bar=Pie(title='各品牌SKU数占比', subtitle="2019-09-22",title_pos='left',title_text_size=22,width=1000,height=800,background_color='white')
bar.add('品牌 VS SKU',x1,y1,is_label_show = True,is_legend_show =False, center=[50, 50], radius=[30, 75], rosetype='radius',label_formatter='{d}')
display(bar)

考拉海淘网上迪奥、圣罗兰等品牌是消费者的主要选择；
而位于最后的KIKO，梦妆，曼秀雷敦则比较小众。

### 热门品牌Top10

In [ ]:
x2=Data.groupby('品牌').mean()['评论数'].sort_values(ascending=False).head(10).index
y2=Data.groupby('品牌').mean().round(2)['评论数'].sort_values(ascending=False).head(10)
bar=Bar(title='热门品牌Top10', subtitle="2019-09-22",title_pos='left',title_text_size=22, title_color='black', width=1000,height=800,background_color='white')
bar.add(name='品牌',x_axis =x2, y_axis = y2,bar_category_gap=5, is_label_show=True, yaxis_name= "评论数均值",xaxis_line_width= 10,xaxis_label_textsize='6',yaxis_label_textsize='9',
xaxis_rotate = 30,yaxis_name_gap=30, is_random= True)
display(bar)

KIKO热度最高，则是因为某一单品评论数高而总的品类数少。

### 热门单品Top10

In [ ]:
x3=Data.groupby('商品标题').max()['评论数'].sort_values(ascending=False).head(10).index
y3=Data.groupby('商品标题').max()['评论数'].sort_values(ascending=False).head(10)
bar=Bar(title='热门单品Top10', subtitle="2019-09-22",title_pos='right',title_text_size=22, title_color='black', width=1200,height=800,background_color='white')
bar.add(name='品牌',x_axis =x3, y_axis = y3,bar_category_gap=5, is_label_show=True, yaxis_name= "最大评论数",xaxis_label_textsize='6',yaxis_label_textsize='9',
xaxis_rotate = 30, is_random= True)
display(bar)

从热门单品TOP10与热门品牌TOP10的结果相似，主要是迪奥、资生堂、MAC。

### 定价

In [ ]:
x4=Data.groupby('品牌').mean().round(2)['discount'].sort_values(ascending=False).index
y4=Data.groupby('品牌').mean().round(2)['discount'].sort_values(ascending=False)
y5=Data.groupby('品牌').mean().round(2)['折扣价_0'].sort_values(ascending=False)
line=Line(title='定价比较', subtitle="2019-09-22",title_pos='left',title_text_size=8, title_color='black', width=1500,height=100,background_color='white')
line.add(name='折扣比例',x_axis =x4, y_axis = y4,mark_line=["average"],is_label_show=True, yaxis_name= "平均折扣比例",yaxis_name_gap=35,
xaxis_rotate = 30)

bar=Bar(title='定价比较', subtitle="2019-09-22",title_pos='left',title_text_size=8, title_color='black', width=1500,height=100,background_color='white')
bar.add(name='折扣价格',x_axis =x4, y_axis = y5,bar_category_gap=5,mark_line=["average"], yaxis_name= "平均折扣价格",yaxis_name_gap=35,is_random= True,
xaxis_rotate = 30)
overlap =Overlap()
overlap.add(bar)

overlap.add(line,yaxis_index=1, is_add_yaxis=True)
display(overlap)

梦妆折扣力度大且价格低，乔治·阿玛尼折扣力度最小且价格高；汤姆·福特定价最高，曼秀雷敦定价最低。

价格高的口红打折力度较小。

### 标签

In [ ]:
df=Data.loc[:,['标签1','标签2','标签3']]
name=df.apply(pd.value_counts).sum(1).index
name.value_counts()

In [ ]:
value=df.apply(pd.value_counts).sum(1)
value.apply(int).sort_values(ascending=False)

In [ ]:
wordcloud = WordCloud(width=500, height=500)
wordcloud.add("", name, value, word_size_range=[20, 100])
display(wordcloud)

白条免息，特价，包税为口红促销常用标签。